In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras import Sequential, layers
from tqdm import tqdm

In [2]:
def format_image(image, label):
    image = tf.image.resize(image, (224,224))/255.0
    return image, label

(train_raw, valid_raw, test_raw), metadata = tfds.load(
    'rock_paper_scissors', split=['train[:80%]','train[80%:]','test'],
    with_info=True, as_supervised=True
)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

BATCH_SIZE = 32
train_batches = train_raw.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
valid_batches = valid_raw.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_raw.map(format_image).batch(1)

for image_batch, label_batch in train_batches.take(1):
    pass

In [4]:
module_selection = ("mobilenet_v2", 224, 1280) 
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

# feature_extractor = hub.KerasLayer(MODULE_HANDLE,
#                                    input_shape=IMAGE_SIZE + (3,), 
#                                    output_shape=[FV_SIZE],
#                                    trainable=False)

feature_extractor = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/3", output_shape=[1280], trainable=False),
  layers.Dense(num_classes, activation='softmax')
])

print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

EPOCHS = 5

hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=valid_batches)


Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280
Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Epoch 1/5
63/63 [==============================] - 12s 54ms/step - loss: 0.9012 - accuracy: 0.6285 - val_loss: 0.8265 - val_accuracy: 0.6587
Epoch 2/5
63/63 [==============================] - 2s 36ms/step - loss: 0.7954 - accuracy: 0.6687 - val_loss: 0.7799 - val_accuracy: 0.6587
Epoch 3/5
63/63 [==============================] - 2s 36ms/step - loss: 0.7552 - accuracy: 0.6687 - val_loss: 0.7476 - val_accuracy: 0.6587
Epoch 4/5
63/63 [==============================] - 2s 36ms/step - loss: 0.7247 - accuracy: 0.6687 - val_loss: 0.7218 - val_accuracy: 0.6587
Epoch 5/5
63/63 [==============================] - 2s 35ms/step - loss: 0.6995 - accuracy: 0.6687 - val_loss: 0.7001 - val_accuracy: 0.6587


In [12]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 3)                 2261827   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 12        
Total params: 2,261,839
Trainable params: 3,855
Non-trainable params: 2,257,984
_________________________________________________________________
